## Some experements

In [1]:
import requests
import bs4

In [2]:
url = "https://lotv.spawningtool.com"
build_pattern = "/build/"

In [3]:
pages = requests.get(url + build_pattern)
pages_parser = bs4.BeautifulSoup(pages.text, "lxml")

In [4]:
builds_data = pages_parser.find('tbody').find_all('tr')
for e in builds_data[:2]:
    params = e.find_all('td')
    print(params[1].find('a').attrs['href'])
    print(params[1].find('a').text)
    print(params[2].text)
    print(params[3].text)
    print("-"*15)

/build/63519/
DnS' 3gate blink pressure into double forge
PvT
Economic
---------------
/build/63104/
Stats' 2 Oracle archon drop into IAC
PvZ
Economic
---------------


In [5]:
builds_data[0].find('a').attrs['href']

'/build/63519/'

In [6]:
slug = "/build/63026/"
url_page = url + slug
url_page

'https://lotv.spawningtool.com/build/63026/'

In [7]:
page_html = requests.get(url_page)
build_parser = bs4.BeautifulSoup(page_html.text, "lxml")
build = build_parser.find("table").find_all('tr')

In [8]:
build[:5]

[<tr><td>  12</td><td>  0:00</td><td>  <nobr><span class="Worker">Drone</span></nobr></td><td>  </td></tr>,
 <tr><td>  13</td><td>  0:13</td><td>  <nobr><span class="Unit">Overlord</span></nobr></td><td>  </td></tr>,
 <tr><td>  13</td><td>  0:19</td><td>  <nobr><span class="Worker">Drone</span></nobr></td><td>  </td></tr>,
 <tr><td>  14</td><td>  0:32</td><td>  <nobr><span class="Worker">Drone x2</span></nobr></td><td>  </td></tr>,
 <tr><td>  16</td><td>  0:50</td><td>  <nobr><span class="Building">Hatchery</span></nobr></td><td>  </td></tr>]

In [9]:
print(build[0].find(class_="Worker") == None)
print(build[0].find(class_="Unit") == None)

False
True


In [10]:
a = build[0]
b = a.find_all('td')
e = b[2]
e.find(class_="Worker")

<span class="Worker">Drone</span>

In [11]:
c = b[1]
d = c.text[2:]#delete spaces before timing
c, d

(<td>  0:00</td>, '0:00')

In [12]:
f = a.find('span')
f.attrs["class"]

['Worker']

In [13]:
for e in build_parser.find_all('ul')[5].find_all('li')[1:]:
    print(e)

<li>Published on: Dec 20, 2017</li>
<li>Modified on: Dec 29, 2017</li>
<li>Patch: 4.0.0</li>


In [14]:
import re
text = build_parser.find_all('ul')[5].find_all('li')[2].text
re.search(r'Modified', text)

<_sre.SRE_Match object; span=(0, 8), match='Modified'>

In [15]:
text[13:]

'Dec 29, 2017'

In [16]:
text = build_parser.find_all('ul')[5].find_all('li')[1].text
re.search(r'Published', text)

<_sre.SRE_Match object; span=(0, 9), match='Published'>

In [17]:
text[14:]

'Dec 20, 2017'

In [18]:
import pandas as pd
df = pd.DataFrame(columns=["title", "build-type", "published-date", "modified-date", "matchup", "build"])
df = df.astype({'title': 'str', 'build-type': 'str', 'matchup': 'str', 'published-date': 'datetime64[ns]', 'modified-date': 'datetime64[ns]'}, copy=False)

In [19]:
df = df.append({'title': 'title1', 'build-type': 'macro', 'modified-date': pd.to_datetime('Dec 23, 2017')}, ignore_index=True)
df

,title,build-type,published-date,modified-date,matchup,build
0,title1,macro,NaT,2017-12-23,NaN,NaN


In [20]:
pd.to_datetime('Dec 23, 2017')

Timestamp('2017-12-23 00:00:00')

In [21]:
df.at[0, "build"] =  {1: 'askd', 2: '23'}
df

,title,build-type,published-date,modified-date,matchup,build
0,title1,macro,NaT,2017-12-23,NaN,"{1: 'askd', 2: '23'}"
